# NSAM Execution demo!

This code below demonstrates how to run the NSAM algorithm on a dataset.

The last cell shows the result of the learning process with the PDDL domain that was learned from the trajectories.

In [8]:
import os
from pathlib import Path
from pddl_plus_parser.lisp_parsers import DomainParser, ProblemParser

from sam_learning.learners import NumericSAMLearner

COUNTERS_DOMAIN_PATH = Path("datasets/counters/counters_domain.pddl")
partial_domain = DomainParser(COUNTERS_DOMAIN_PATH, partial_parsing=True).parse_domain()

nsam = NumericSAMLearner(partial_domain=partial_domain)

In [9]:
from pddl_plus_parser.lisp_parsers import TrajectoryParser

WORKDIR_PATH = Path("datasets/counters")

os.environ["CONVEX_HULL_ERROR_PATH"] = str(WORKDIR_PATH / "convex_hull_error.txt")

allowed_observations = []
sorted_trajectory_paths = sorted(WORKDIR_PATH.glob("*.trajectory"))  # for consistency
for index, trajectory_file_path in enumerate(sorted_trajectory_paths):
    problem_path = WORKDIR_PATH / f"{trajectory_file_path.stem}.pddl"
    problem = ProblemParser(problem_path, partial_domain).parse_problem()
    complete_observation = TrajectoryParser(partial_domain, problem).parse_trajectory(trajectory_file_path)
    allowed_observations.append(complete_observation)
    if len(allowed_observations) >= 10:
        break

learned_model, learning_report = nsam.learn_action_model(allowed_observations)

In [10]:
print(learned_model.to_pddl())

(define (domain fo-counters-rnd)
(:requirements :strips :typing :equality :adl :fluents :negative-preconditions)
(:types 	counter - object
)

(:predicates (nop ?x - counter)
)

(:functions (value ?c - counter)
	(rate_value ?c - counter)
	(max_int )
)

(:action increment
	:parameters (?c - counter)
	:precondition (and (not (nop ?c))
	(<= (* (rate_value ?c) -1) -1)
	(<= (+ (* (rate_value ?c) -0.3714) (* (value ?c) -0.9285)) -4.2710)
	(<= (+ (* (rate_value ?c) 0.4472) (* (value ?c) 0.8944)) 86.7594)
	(<= (+ (* (rate_value ?c) 0.9975) (* (value ?c) -0.0712)) 8.9059)
	(<= (+ (* (value ?c) -0.1240) (* (rate_value ?c) -0.9923)) -4.3412)
	(<= (+ (* (value ?c) 0.0200) (* (rate_value ?c) -0.9998)) -0.1800)
	(<= (+ (* (value ?c) 0.2425) (* (rate_value ?c) -0.9701)) 20.1305)
	(<= (+ (* (value ?c) 0.6247) (* (rate_value ?c) 0.7809)) 63.4065)
	(<= (rate_value ?c) 10)
	(<= (value ?c) 95)
	(= (max_int ) 100))
	:effect (and 
(increase (value ?c) (rate_value ?c))))

(:action decrement
	:parameters (?c -